# 1. 트랜스포머 모델의 개괄

### 1.1. 트랜스포머란?


왼쪽이 디코더(Decoder) 레이어, 오른쪽이 인코더(Encoder) 레이어

셀프 어텐션과 멀티 헤드 어텐션을 수행함

# 2. 어텐션 메커니즘(Attention Mechanism)



그래서 트랜스포머 구조를 이해하기 위해 **어텐션 메커니즘**을 이해할 필요가 있음

CNN을 알기위해 Convolution을 알아야 하는것 처럼



### 2.1. 기본 아이디어

디코더에서 출력 예측하는 시점마다 인코더의 전체 입력을 다시 참고함

여기서 전체 입력 중 예측하는 부분과 연관된 부분에 더 집중(Attenrion)해서 보기 때문에 

    어텐션 메커니즘(Attention Mechanism)

## 2.2 구성요소


    쿼리(Query), 키(Key), 벨류(Value)

위의 3가지 구성요소는 입력데이터($x$)에 대한 "선형변환" **(벡터와 행렬의 곱) $ = [vector][matrix]$** 으로 생성됨
$$Q(Query) = xW_Q$$
$$K(Key) = xW_K$$
$$V(Value) = xW_V$$

$W_Q,W_K,W_V$는 가중치 행렬이며 모델이 학습하며 업데이트하는 값

$W_Q,W_K,W_V$는 임베딩 차원($d_{model}$)과 헤드의 개수($num\;head$)에 기반해 생성

임베딩 차원과 헤드가 뭔지는 트랜스포머 모델을 설명할 때


### 2.2.1 임베딩
입력 데이터에 대한 고차원의 연속적인 특징 공간
"The cat sat on the mat"를 입력 데이터라 할 때 $d_{model} = 512$ 라고 하자

입력데이터에 대한 임베딩 결과는 $x.shape = [6,512] = [입력데이터의 길이,d_{model}]$


### 2.2.2 멀티-헤드

트렌스포머 모델은 입력 데이터를 여러 헤드에서 동시에 처리함
각 헤드마다 다른 관점에서 분석

입력 데이터에 임베딩에는 사용되지 않음

쿼리,키,벨류의 가중치 행렬 생성시 $shape = [d_{model},{d_{model} \over num\;head}]$으로 생성함

## 2.3 쿼리(Query)

현재 단어를 나타내는 벡터를 의미함

가중치 행렬 $W_Q = $

$$W_Q = \begin{bmatrix}
q_{11} & \cdots & q_{1(num\;head)}\\
\vdots & \ddots & \vdots \\
q_{(d_{model})1} & \cdots & q_{(d_{model})(num\;head)}\\
\end{bmatrix}$$

가중치 행렬의 크기
$$W_Q.shape = [d_{model},{d_{model} \over num\;head}]$$


#### 예시
"The cat sat on the mat"에서 "cat"이라는 단어를 분석한다고 가정할 때,$d_{model} = 512,num\;head=8 $ 이라고 하자

 "cat"에 대응되는 **쿼리 벡터**를 찾고싶다.



입력 데이터의 임베딩($x$)를 $x = \begin{bmatrix}x_{1,1} & \cdots & x_{1,512}\\
\vdots & \ddots & \vdots \\
x_{6,1} & \cdots & x_{6,512}\end{bmatrix}$라 하자,

 임의의 $W_Q = \begin{bmatrix}
q_{1,1} & \cdots & q_{1,64}\\
\vdots & \ddots & \vdots \\
q_{512,1} & \cdots & q_{512,64}\\
\end{bmatrix}$가 있을 때,
 
  선형변환($=xW_Q$) 된 결과
  
$$Q =
\begin{bmatrix}x_{1,1} & \cdots & x_{1,512}\\
\vdots & \ddots & \vdots \\
x_{6,1} & \cdots & x_{6,512}\end{bmatrix}
\begin{bmatrix}
q_{1,1} & \cdots & q_{1,64}\\
\vdots & \ddots & \vdots \\
q_{512,1} & \cdots & q_{512,64}\\
\end{bmatrix} 
=\begin{bmatrix}
(x_{1,1})(q_{1,1}) +\cdots + (x_{1,512})(q_{512,1})& \cdots & (x_{1,1})(q_{1,64}) +\cdots +(x_{1,512}q_{512,64}) \\
\vdots & \ddots & \vdots \\
(x_{6,1})(q_{1,1}) +\cdots + (x_{1,512})(q_{512,1})& \cdots & (x_{6,1})(q_{1,64}) +\cdots + (x_{6,512})(q_{512,64})\\
\end{bmatrix}$$

$$Q.shape = [6,64] = [입력데이터길이,{d_{model} \over num\;head}]$$

$$\begin{bmatrix} (x_{2,1})(q_{1,2}) +\cdots + (x_{2,512})(q_{512,2}) \\ \vdots \\ (x_{2,1})(q_{1,1}) +\cdots + (x_{2,512})(q_{512,1})\end{bmatrix}\;이\;"cat"에\;대한\;쿼리 벡터\; 이다.$$

## 2.4 키(Key) - 벨류(Value)

- 기본개념: 키(Key) - 벨류(Value) 구조는 흔히 아래와 같은 딕셔너리 형에서 사용됨

    키값으로 벨류를 가져올 수 있다.


In [4]:
dict = {"2017" : "Transformer", "2018" : "BERT"}
print(dict["2018"])

BERT


키(Key)가중치 행렬 

$$W_K = \begin{bmatrix}
k_{11} & \cdots & k_{1(num\;head)}\\
\vdots & \ddots & \vdots \\
k_{(d_{model})1} & \cdots & k_{(d_{model})(num\;head)}\\
\end{bmatrix}$$

가중치 행렬의 크기
$$W_K.shape = [d_{model},{d_{model} \over num\;head}]$$

벨류(Value)가중치 행렬 

$$W_V = \begin{bmatrix}
v_{11} & \cdots & v_{1(num\;head)}\\
\vdots & \ddots & \vdots \\
v_{(d_{model})1} & \cdots & v_{(d_{model})(num\;head)}\\
\end{bmatrix}$$

가중치 행렬의 크기
$$W_K.shape = [d_{model},{d_{model} \over num\;head}]$$

#### 쿼리,키,벨류의 가중치 행렬은 모두 같은 크기를 가진다. 

## 2.5 Attention Score 계산

입력 시퀀스(=현재 쿼리)의 대한 전체요소들과의 가중치를 계산하는 것이다.

이 계산 방법에 따라 다양한 방법으로 구현 될 수 있다. 그 중 내적(Dot-Product)을 이용한 계산을 알아보자면
$$ Attention(Q,K,V) = softmax({QK^T \over \sqrt{d_k}}) $$
로 정의되며 싱글 헤드 어텐션과 멀티 헤드 어텐션일때 계산이 달라진다.

 - **멀티-헤드 어텐션**
$$ Attention(Q,K,V) = softmax({QK^T \over \sqrt{d_k}}) = softmax({QK^T \over \sqrt{{d_{model} \over num\;head}}})$$

 - 싱글-헤드셀프 어텐션(별로 중요하지 않아보인다. 이때는 위의 가중치 행렬의 모양또한 바뀔것이다.) 
 $$ = softmax({QK^T \over \sqrt{d_{model}}}) = softmax({QK^T \over \sqrt{{d_{model} \over 1}}})$$



# 3. 트랜스포머 모델

앞서 설명한 어텐션 개념만으로 작동함

## 3.1. 임베딩 차원과 멀티 헤드 어텐션

In [2]:
import torch
import torch.nn.functional as F

def scaled_dot_product_attention(query, key, value, mask=None):
    """
    어텐션 점수, 가중치, 그리고 출력을 계산합니다.
    
    Args:
    - query (torch.Tensor): 쿼리 텐서, 크기는 (batch_size, num_queries, d_k).
    - key (torch.Tensor): 키 텐서, 크기는 (batch_size, num_keys, d_k).
    - value (torch.Tensor): 값 텐서, 크기는 (batch_size, num_keys, d_v).
    - mask (torch.Tensor, optional): 마스크 텐서, 크기는 (batch_size, num_queries, num_keys).
    
    Returns:
    - output (torch.Tensor): 출력 텐서, 크기는 (batch_size, num_queries, d_v).
    - attention_weights (torch.Tensor): 어텐션 가중치, 크기는 (batch_size, num_queries, num_keys).
    """
    
    # 1. 어텐션 점수 계산
    # 쿼리와 키 간의 내적을 계산합니다.
    attention_scores = torch.matmul(query, key.transpose(-2, -1))  # (batch_size, num_queries, num_keys)
    
    # 어텐션 점수를 스케일링합니다.
    d_k = key.size(-1)
    attention_scores = attention_scores / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
    
    # 제공된 경우 마스크를 적용합니다.
    if mask is not None:
        attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))
    
    # 2. 어텐션 가중치 계산
    # 어텐션 점수에 소프트맥스 함수를 적용하여 어텐션 가중치를 얻습니다.
    attention_weights = F.softmax(attention_scores, dim=-1)  # (batch_size, num_queries, num_keys)
    
    # 3. 출력 계산
    # 값의 가중 평균을 계산하여 출력을 얻습니다.
    output = torch.matmul(attention_weights, value)  # (batch_size, num_queries, d_v) , 행렬의 곱셈 = 행렬의 내적
    
    return output, attention_weights

# 예제 사용:
batch_size = 32
num_queries = 10
num_keys = 100
d_k = 64
d_v = 32

query = torch.randn(batch_size, num_queries, d_k)
print("Shape of Q:",query.shape)

#키와 쿼리의 개수는 동일,벨류의 개수는 다름
#Q,K,V 는 모두 같은 인풋(임베딩 됨)에서 나옴, but가중치 행렬이 적용되어 사이즈가 달라짐
key = torch.randn(batch_size, num_keys, d_k)
print("Shape of K:",key.shape)
value = torch.randn(batch_size, num_keys, d_v)
print("Shape of V:",value.shape)

output, attention_weights = scaled_dot_product_attention(query, key, value)
print(output.shape)  # 예상: torch.Size([32, 10, 64])
print(attention_weights.size)  # 예상: torch.Size([32, 10, 100])
print((attention_weights[0][:][:]))  

Shape of Q: torch.Size([32, 10, 64])
Shape of K: torch.Size([32, 100, 64])
Shape of V: torch.Size([32, 100, 32])
torch.Size([32, 10, 32])
<built-in method size of Tensor object at 0x000001C2E068C640>
tensor([[0.0023, 0.0037, 0.0115, 0.0196, 0.0101, 0.0025, 0.0063, 0.0114, 0.0012,
         0.0053, 0.0053, 0.0054, 0.0004, 0.0393, 0.0050, 0.0042, 0.0126, 0.0063,
         0.0117, 0.0041, 0.0088, 0.0091, 0.0198, 0.0061, 0.0021, 0.0101, 0.0032,
         0.0005, 0.0024, 0.0013, 0.0033, 0.0163, 0.0086, 0.0063, 0.0048, 0.0636,
         0.0076, 0.0051, 0.0042, 0.0061, 0.0032, 0.0054, 0.0063, 0.0010, 0.0369,
         0.0032, 0.0040, 0.0061, 0.0149, 0.0032, 0.0052, 0.0087, 0.0046, 0.0399,
         0.0031, 0.0144, 0.0013, 0.0353, 0.0086, 0.0003, 0.0014, 0.0055, 0.0063,
         0.0044, 0.0023, 0.0091, 0.0021, 0.0012, 0.0039, 0.0006, 0.0090, 0.0156,
         0.0060, 0.0131, 0.0112, 0.0231, 0.0057, 0.0048, 0.0038, 0.0030, 0.0017,
         0.0873, 0.0009, 0.0035, 0.0157, 0.0085, 0.0055, 0.0025, 0.0069

In [3]:
import torch
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, nhead, num_layers, max_len, dropout=0.5):
        super(TransformerModel, self).__init__()
        
        # 소스 단어 임베딩
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        # 타겟 단어 임베딩
        self.trg_embedding = nn.Embedding(trg_vocab_size, d_model)
        # 위치 인코딩 생성
        self.positional_encoding = self._generate_positional_encoding(d_model, max_len)
        
        # Transformer 모델 정의
        self.transformer = nn.Transformer(d_model, nhead, num_layers, num_layers, dim_feedforward=d_model*4, dropout=dropout)
        # 출력 레이어
        self.fc_out = nn.Linear(d_model, trg_vocab_size)
        
    def _generate_positional_encoding(self, d_model, max_len):
        # 위치 인코딩 행렬 생성
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        return pe

    def forward(self, src, trg):
        # 소스 단어 임베딩과 위치 인코딩을 더함
        src_emb = self.src_embedding(src) + self.positional_encoding[:src.size(0), :]
        # 타겟 단어 임베딩과 위치 인코딩을 더함
        trg_emb = self.trg_embedding(trg) + self.positional_encoding[:trg.size(0), :]
        
        # Transformer 모델의 forward 연산
        output = self.transformer(src_emb, trg_emb)
        # 출력 레이어를 통과시킴
        return self.fc_out(output)

# 예제
SRC_VOCAB_SIZE = 9998
TRG_VOCAB_SIZE = 9998
D_MODEL = 512
NHEAD = 8
NUM_LAYERS = 6
MAX_LEN = 100

# TransformerModel 인스턴스 생성
model = TransformerModel(SRC_VOCAB_SIZE, TRG_VOCAB_SIZE, D_MODEL, NHEAD, NUM_LAYERS, MAX_LEN)
print(model)


TransformerModel(
  (src_embedding): Embedding(9998, 512)
  (trg_embedding): Embedding(9998, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.5, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.5, inplace=False)
          (dropout2): Dropout(p=0.5, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
  